<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=bf5a5e9d40ed5ca4fd69e7f6a1758b7a5076f1fe0e380f0aec6ae87cc58e38eb
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-01-01 14:29:41
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 14
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.68 K
Current:  1.43 C
-------------------
Today PnL: 27.19 K (0.19%)
Current PnL: -25.67 L (-16.37%)
CY Booked + Current PnL: -11.38 L (-7.26%)
-------------------
Total profit:  1.62 L
Total loss:  -27.29 L
-------------------
Total Booked + Current PnL: 15.60 L (12.01%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (9.96%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.34 C
Est FTT PnL: 90.64 L (63.21%)
Deployed:  1.30 C
Current:  1.43 C
CAGR/XIRR %: 5.5%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,NESTLEIND,1377.00,0.47,14.09,6.41,21.40,19411.0,37393.0,302819.0,5.82,68.0,X-LC,5.15,12.0,1.93,2.11,21.31,XY25,NTT,FMCG
77,TTKPRESTIG,770.00,-0.17,-19.94,25.02,0.09,20187.0,-20094.0,80683.0,87.19,38.0,M-SC,3.37,253.0,-1.00,0.56,6.00,OX40N,NTT,DURABLES
82,VOLTAS,1530.00,1.64,8.23,10.59,19.69,21977.0,15783.0,207525.0,-2.32,53.0,X-MC,2.08,78.0,0.72,1.45,15.59,XY25,NTT,AC
49,MASFIN,398.61,-1.67,-2.57,25.27,22.05,24123.0,-2520.0,95460.0,-15.99,52.0,H-SC,6.22,164.0,-0.10,0.67,38.76,XR,ATH,FINANCE
39,INDIGOPNTS,1408.00,3.48,-15.92,18.96,0.02,27827.0,-27793.0,146766.0,123.77,50.0,M-SC,7.05,234.0,-1.00,1.02,27.66,OX40N,NTT,PAINTS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,11.01,-29.44,60.01,12.90,92517.0,-64326.0,154170.0,127.16,71.0,H-SC,11.79,161.0,-0.70,1.08,28.19,OX40N,NTT,FINANCE
6,ATULAUTO,844.00,7.57,-18.40,78.40,45.57,126110.0,-36278.0,160854.0,3483.78,65.0,M-SC,6.45,244.0,-0.29,1.12,14.04,XY24,NTT,AUTO
25,FINCABLES,1641.55,4.94,-0.36,108.66,107.91,159853.0,-531.0,147113.0,-18.93,58.0,M-SC,11.32,216.0,-0.00,1.03,9.69,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,3.48,-15.92,18.96,0.02,27827.0,-27793.0,146766.0,123.77,50.0,M-SC,7.05,234.0,-1.00,1.02,27.66,OX40N,NTT,PAINTS
60,REPCOHOME,880.00,2.92,-7.91,106.11,89.82,279977.0,-22650.0,263855.0,-53.80,64.0,H-SC,5.17,160.0,-0.08,1.84,39.60,XY24,NTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-9.09,-10.21,23.38,10.78,50449.0,-24538.0,215780.0,-46.47,19.0,X-LC,2.44,5.0,-0.49,1.50,0.00,X40,NTT,FMCG
78,UNITDSPR,1644.00,-2.44,3.47,16.72,20.77,40506.0,8118.0,242262.0,-5.50,45.0,X-MC,1.47,62.0,0.20,1.69,10.63,X40N,NTT,BREWERIES
26,GILLETTE,11206.78,-2.42,-3.58,37.34,32.42,91408.0,-9096.0,244800.0,-20.76,45.0,X-SC,11.60,85.0,-0.10,1.71,10.98,X40,ATH,FMCG
2,ABBOTINDIA,35195.00,-1.79,-4.38,23.47,18.07,40141.0,-7825.0,171030.0,-18.20,47.0,X-MC,7.66,63.0,-0.19,1.19,13.33,X40,ATH,PHARMA
49,MASFIN,398.61,-1.67,-2.57,25.27,22.05,24123.0,-2520.0,95460.0,-15.99,52.0,H-SC,6.22,164.0,-0.10,0.67,38.76,XR,ATH,FINANCE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,4.94,-0.36,108.66,107.91,159853.0,-531.0,147113.0,-18.93,58.0,M-SC,11.32,216.0,-0.00,1.03,9.69,OX40N,ATH,CABLES
57,RECLTD,446.00,2.54,-0.96,21.91,20.74,44087.0,-1952.0,201218.0,31.15,64.0,M-MC,8.27,189.0,-0.04,1.40,9.55,XY25,NTT,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,4.94,-0.36,108.66,107.91,159853.0,-531.0,147113.0,-18.93,58.0,M-SC,11.32,216.0,-0.00,1.03,9.69,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,3.48,-15.92,18.96,0.02,27827.0,-27793.0,146766.0,123.77,50.0,M-SC,7.05,234.0,-1.00,1.02,27.66,OX40N,NTT,PAINTS
47,KANSAINER,340.00,1.93,-19.90,41.67,13.47,90007.0,-53667.0,216000.0,-67.15,60.0,H-SC,6.29,158.0,-0.60,1.51,9.83,XY24,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.17,-19.94,25.02,0.09,20187.0,-20094.0,80683.0,87.19,38.0,M-SC,3.37,253.0,-1.00,0.56,6.00,OX40N,NTT,DURABLES
66,SIS,528.00,-0.36,-23.71,58.94,21.26,49929.0,-26321.0,84711.0,1998.96,50.0,H-SC,4.71,166.0,-0.53,0.59,14.45,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,1.47,9.57,57.22,72.27,106240.0,16221.0,185669.0,-6.08,62.0,M-LC,5.43,99.0,0.15,1.29,16.34,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,1.47,9.57,57.22,72.27,106240.0,16221.0,185669.0,-6.08,62.0,M-LC,5.43,99.0,0.15,1.29,16.34,XR,NTT,IT
25,FINCABLES,1641.55,4.94,-0.36,108.66,107.91,159853.0,-531.0,147113.0,-18.93,58.0,M-SC,11.32,216.0,-0.00,1.03,9.69,OX40N,ATH,CABLES
38,INDIAMART,4810.62,-1.23,-5.63,119.05,106.72,138566.0,-6943.0,116393.0,-53.30,39.0,H-SC,6.09,139.0,-0.05,0.81,16.43,AR,ATH,MISC
85,ZYDUSLIFE,1286.17,0.11,-3.47,40.51,35.63,81948.0,-7282.0,202292.0,-17.18,44.0,H-MC,2.82,119.0,-0.09,1.41,12.62,AR,ATH,PHARMA
49,MASFIN,398.61,-1.67,-2.57,25.27,22.05,24123.0,-2520.0,95460.0,-15.99,52.0,H-SC,6.22,164.0,-0.10,0.67,38.76,XR,ATH,FINANCE


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-9.09,-10.21,23.38,10.78,50449.0,-24538.0,215780.0,-46.47,19.0,X-LC,2.44,5.0,-0.49,1.50,0.00,X40,NTT,FMCG
22,DIXON,18931.72,-1.65,-17.48,59.06,31.26,112469.0,-40336.0,190432.0,-61.45,29.0,X-MC,16.63,56.0,-0.36,1.33,1.15,X40N,ATH,IT
34,HONAUT,58357.33,0.52,-19.02,76.97,43.32,101523.0,-30972.0,131900.0,-30.40,31.0,X-SC,7.51,90.0,-0.31,0.92,1.27,X40N,ATH,ELECTRICAL
23,DMART,5391.45,-1.62,-10.55,44.89,29.60,100221.0,-26340.0,223260.0,-23.73,32.0,X-LC,8.89,19.0,-0.26,1.56,9.31,X40N,ATH,FMCG
52,PAGEIND,51605.08,-1.23,-9.89,44.96,30.62,96035.0,-23454.0,213600.0,-35.98,34.0,X-MC,15.33,55.0,-0.24,1.49,0.10,X40,ATH,APPARELS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.5,0.30,-17.45,52.05,25.51,79959.0,-32475.0,153620.0,-5.62,42.0,H-LC,0.90,49.0,-0.41,1.07,13.21,AR,ATH,ELECTRICAL
78,UNITDSPR,1644.0,-2.44,3.47,16.72,20.77,40506.0,8118.0,242262.0,-5.50,45.0,X-MC,1.47,62.0,0.20,1.69,10.63,X40N,NTT,BREWERIES
12,BERGEPAINT,680.0,-0.84,-5.22,27.81,21.14,59925.0,-11859.0,215480.0,-11.94,42.0,X-MC,1.82,74.0,-0.20,1.50,19.66,XY24,NTT,PAINTS
20,DABUR,735.0,-0.88,-2.41,47.25,43.70,114387.0,-5980.0,242088.0,-10.52,49.0,X-MC,1.97,72.0,-0.05,1.69,12.28,XY24,BTT,FMCG
82,VOLTAS,1530.0,1.64,8.23,10.59,19.69,21977.0,15783.0,207525.0,-2.32,53.0,X-MC,2.08,78.0,0.72,1.45,15.59,XY25,NTT,AC


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-9.09,-10.21,23.38,10.78,50449.0,-24538.0,215780.0,-46.47,19.0,X-LC,2.44,5.0,-0.49,1.50,0.00,X40,NTT,FMCG
52,PAGEIND,51605.08,-1.23,-9.89,44.96,30.62,96035.0,-23454.0,213600.0,-35.98,34.0,X-MC,15.33,55.0,-0.24,1.49,0.10,X40,ATH,APPARELS
3,ACC,3906.00,0.19,-26.83,124.35,64.15,216493.0,-63851.0,174100.0,-57.19,37.0,X-SC,5.80,82.0,-0.29,1.21,0.92,XY24,BTT,CEMENT
11,BATAINDIA,2096.00,0.53,-38.85,121.10,35.20,95286.0,-49986.0,78684.0,3.10,39.0,X-SC,17.32,92.0,-0.52,0.55,1.06,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-1.65,-17.48,59.06,31.26,112469.0,-40336.0,190432.0,-61.45,29.0,X-MC,16.63,56.0,-0.36,1.33,1.15,X40N,ATH,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,424.00,1.46,-29.73,103.27,42.84,47175.0,-19325.0,45681.0,-53.72,45.0,X-SC,31.12,83.0,-0.41,0.32,3.35,XY24,NTT,MISC
58,RELAXO,1176.00,-0.51,-47.89,192.28,52.31,145444.0,-69518.0,75642.0,-44.54,43.0,X-SC,7.42,91.0,-0.48,0.53,1.25,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.53,-38.85,121.10,35.20,95286.0,-49986.0,78684.0,3.10,39.0,X-SC,17.32,92.0,-0.52,0.55,1.06,X40,NTT,FOOTWEAR
50,MEDANTA,1486.00,0.47,-0.47,24.66,24.08,31158.0,-598.0,126352.0,-8.54,52.0,X-SC,7.63,89.0,-0.02,0.88,19.05,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,0.52,-19.02,76.97,43.32,101523.0,-30972.0,131900.0,-30.40,31.0,X-SC,7.51,90.0,-0.31,0.92,1.27,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,0.66,-11.26,36.03,20.71,120931.0,-42598.0,335639.0,-23.62,51.0,X-LC,2.92,1.0,-0.35,2.34,12.15,X40,ATH,IT
41,INFY,1972.00,1.18,11.27,20.65,34.24,71893.0,35260.0,348148.0,-13.50,57.0,X-LC,7.43,2.0,0.49,2.43,20.49,X40,NTT,IT
75,TMPV,600.00,-0.38,-24.99,63.96,22.98,108371.0,-56463.0,169435.0,-21.86,57.0,X-LC,5.71,3.0,-0.52,1.18,6.57,XY24,NTT,AUTO
81,VBL,671.64,0.41,-0.65,36.55,35.67,114694.0,-2042.0,313800.0,-12.49,64.0,X-LC,2.80,4.0,-0.02,2.19,13.12,X40N,ATH,FMCG
43,ITC,452.00,-9.09,-10.21,23.38,10.78,50449.0,-24538.0,215780.0,-46.47,19.0,X-LC,2.44,5.0,-0.49,1.50,0.00,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,2.56,-34.57,103.09,32.89,52080.0,-26688.0,50519.0,-723.88,67.0,L-MC,6.70,259.0,-0.51,0.35,39.13,XR,NTT,BANKS
49,MASFIN,398.61,-1.67,-2.57,25.27,22.05,24123.0,-2520.0,95460.0,-15.99,52.0,H-SC,6.22,164.0,-0.10,0.67,38.76,XR,ATH,FINANCE
13,BSOFT,831.70,-0.28,-21.66,92.48,50.79,101900.0,-30467.0,110186.0,0.50,54.0,H-SC,10.82,151.0,-0.30,0.77,28.29,XR,ATH,IT
39,INDIGOPNTS,1408.00,3.48,-15.92,18.96,0.02,27827.0,-27793.0,146766.0,123.77,50.0,M-SC,7.05,234.0,-1.00,1.02,27.66,OX40N,NTT,PAINTS
69,SURYODAY,216.00,1.52,-18.55,51.06,23.03,74471.0,-33221.0,145850.0,57.89,56.0,H-SC,6.59,167.0,-0.45,1.02,44.71,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.0,11.01,-29.44,60.01,12.90,92517.0,-64326.0,154170.0,127.16,71.0,H-SC,11.79,161.0,-0.70,1.08,28.19,OX40N,NTT,FINANCE
13,BSOFT,831.7,-0.28,-21.66,92.48,50.79,101900.0,-30467.0,110186.0,0.50,54.0,H-SC,10.82,151.0,-0.30,0.77,28.29,XR,ATH,IT
36,ICICIPRULI,790.0,0.76,11.42,17.32,30.73,38019.0,22507.0,219512.0,-14.61,72.0,X-MC,4.77,75.0,0.59,1.53,25.65,X40,ATH,INSURANCE
74,TITAGARH,1548.0,-1.07,-16.93,75.43,45.74,151755.0,-40990.0,201187.0,10.76,61.0,H-SC,4.43,174.0,-0.27,1.40,30.85,XY24,NTT,ENGINEERING
84,WIPRO,420.0,1.47,9.57,57.22,72.27,106240.0,16221.0,185669.0,-6.08,62.0,M-LC,5.43,99.0,0.15,1.29,16.34,XR,NTT,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.08
1,25,44.09
2,50,76.39


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.87
MC    30.75
LC    26.35
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.74
X40      23.90
X40N     12.81
XY25     10.49
XR        8.97
AR        8.55
OX40N     7.53
SR        0.98
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.31
X-LC    22.95
H-SC    22.81
M-SC    11.43
X-SC     7.82
H-MC     4.69
M-MC     1.40
M-LC     1.29
H-LC     1.07
L-LC     1.04
L-SC     0.81
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.04,-7.77,43.13
IT,12.75,-16.90,75.86
FINANCE,11.04,-12.72,60.03
MISC,6.96,-30.08,83.28
ELECTRICAL,5.79,-12.96,54.34
PAINTS,5.73,-12.36,29.05
INSURANCE,4.73,0.14,34.02
PHARMA,3.97,-3.04,35.48
AUTO,2.88,-26.78,70.74


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3134162.0,21
AR,1344545.0,10
XR,1205190.0,12
X40,1091162.0,15
X40N,907060.0,9
OX40N,712633.0,10
XY25,389201.0,7
SR,280280.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3524749.0,24
X-MC,1497071.0,16
M-SC,1418016.0,15
X-LC,932390.0,13
X-SC,798682.0,8
H-MC,406048.0,3
L-SC,173924.0,2
M-LC,106240.0,1
H-LC,79959.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1224702.0      6
           AR         921547.0      5
M-SC       XY24       802476.0      6
H-SC       XR         782963.0      7
X-MC       X40        502110.0      7
           XY24       434383.0      3
X-LC       X40        402358.0      6
X-MC       X40N       374983.0      4
X-SC       X40N       317162.0      3
H-SC       OX40N      315257.0      4
M-SC       OX40N      308070.0      5
X-SC       XY24       294826.0      3
H-SC       SR         280280.0      2
X-LC       X40N       214915.0      2
H-MC       AR         214858.0      2
           XY24       191190.0      1
X-SC       X40        186694.0      2
X-LC       XY24       186585.0      2
X-MC       XY25       185595.0      2
M-SC       XR         179289.0      2
X-LC       XY25       128532.0      3
M-SC       AR         128181.0      2
M-LC       XR         106240.0      1
L-SC       OX40N       89306.0      1
           XR          84618.0      1
H-LC       AR          79959.0      1
L-MC       XR          52080.0      1
M-MC       XY25        44087.0      1
L-LC       XY25        30987.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 32.0 seconds
